In [106]:
from pystac_client import Client
from odc.stac import load
import odc.geo
import xarray as xr
import rioxarray as rio
import geopandas as gpd 
import itertools
from multiprocessing import Pool
import dask.distributed
import dask.utils
from odc.stac import configure_rio, stac_load
from odc.algo import erase_bad, mask_cleanup
import numpy as np

CLOUD_SHADOWS = 3
CLOUD_HIGH_PROBABILITY = 9
NO_DATA = 0

bitmask_cloud = 0
for field in [CLOUD_SHADOWS, CLOUD_HIGH_PROBABILITY]:
    bitmask_cloud |= 1 << field

bitmask_nodata = 0
for field in [NO_DATA]:
    bitmask_nodata |= 1 << field


# Start dask cluster
client = dask.distributed.Client()
configure_rio(cloud_defaults=True, client = client)

# Prepare region of interest
spain = gpd.read_file('/Users/diegobengochea/git/iberian.carbon/data/SpainPolygon/gadm41_ESP_1.shp')
#spain = gpd.read_file('/home/dibepa/git/iberian.carbon/spain/gadm41_ESP_1.shp')
# Filter continental Spain
spain = spain[ (spain.GID_1 != 'ESP.7_1') & (spain.GID_1 != 'ESP.13_1') & (spain.GID_1 != 'ESP.14_1') ] 
# Reproject to UTM30 EPSG:25830
spain = spain.dissolve()[['geometry','COUNTRY']].to_crs(epsg='25830')
# Add CRS information to shapely polygon
geometry_spain = odc.geo.geom.Geometry(spain.geometry[0],crs='EPSG:25830')

# Create a GeoBox for all continental Spain with a 10 meters resolution 
geobox_spain = odc.geo.geobox.GeoBox.from_geopolygon(geometry_spain,resolution=10) # The resolution here is irrelevant since the Spain GeoBOX is too large to make queries, adn thus cannot be used as intersect. Only used to create tiles of suitable shape 20km2



/opt/homebrew/Caskroom/miniforge/base/envs/sentinel.mosaics/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 60764 instead
  warnings.warn(


In [107]:
# Divide the full geobox in Geotiles of smaller size for processing
geotiles_spain = odc.geo.geobox.GeoboxTiles(geobox_spain,(100,100))

geotiles_spain = [ geotiles_spain.__getitem__(tile) for tile in geotiles_spain._all_tiles() ]

In [17]:
len(geotiles_spain)

914788

In [108]:
geobox = geotiles_spain[110000]
year = 2018

bbox = geobox.boundingbox.to_crs('EPSG:4326')
bbox = (bbox.left,bbox.bottom,bbox.right,bbox.top)

green_season = f'{year}-04-01/{year}-05-01'

catalog = Client.open("https://earth-search.aws.element84.com/v1") 
search = catalog.search(
    collections = ['sentinel-2-l2a'],
    bbox = bbox, 
    datetime = green_season,
    query = ['eo:cloud_cover<50']
)

item_collection = search.item_collection()
print(len(item_collection))



2


In [19]:
geobox.boundingbox.to_crs('epsg:4326')

BoundingBox(left=-2.6329613682457893, bottom=43.01382645884041, right=-2.62063475983079, top=43.02287134535306, crs=CRS('EPSG:4326'))

In [55]:
item_collection.save_object('sentinel2-items.json')

In [134]:
src_dataset = odc.stac.load(
    item_collection,
    bands = ['red', 'green', 'blue', 'nir', 'swir16', 'swir22', 'rededge1', 'rededge2', 'rededge3', 'nir08','scl'],
    geobox = geobox,
    chunks = {'x':10,'y':10},
    groupby = 'solar_day',
    resampling = 'bilinear'
)

#src_dataset[['red', 'green', 'blue', 'nir', 'swir16', 'swir22', 'rededge1', 'rededge2', 'rededge3', 'nir08']] = src_dataset[['red', 'green', 'blue', 'nir', 'swir16', 'swir22', 'rededge1', 'rededge2', 'rededge3', 'nir08']].astype('float32')

cloud_mask = src_dataset.scl.astype("uint16") & bitmask_cloud != 0
cloud_mask = mask_cleanup(cloud_mask) # Use default filters
src_dataset = erase_bad(src_dataset, cloud_mask)

src_dataset = src_dataset.where(~cloud_mask)
src_dataset = src_dataset.where(~nodata_mask)

src_dataset

<xarray.Dataset> Size: 422kB
Dimensions:      (y: 100, x: 100, time: 2)
Coordinates:
  * y            (y) float64 800B 4.753e+06 4.753e+06 ... 4.752e+06 4.752e+06
  * x            (x) float64 800B 1.699e+05 1.699e+05 ... 1.709e+05 1.709e+05
    spatial_ref  int32 4B 25830
  * time         (time) datetime64[ns] 16B 2018-04-20T11:26:11.293000 2018-04...
Data variables:
    red          (time, y, x) uint16 40kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    green        (time, y, x) uint16 40kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    blue         (time, y, x) uint16 40kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    nir          (time, y, x) uint16 40kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    swir16       (time, y, x) uint16 40kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    swir22       (time, y, x) uint16 40kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    rededge1     (time, y, x) uint16 40kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    rededge2     (time, y, x) uint16 40kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    rededge3     (time, y, x) uint16 40kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    nir08        (time, y, x) uint16 40kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    scl          (time, y, x) uint8 20kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>

In [135]:
data = src_dataset[['blue','scl']].compute()
data

<xarray.Dataset> Size: 62kB
Dimensions:      (time: 2, y: 100, x: 100)
Coordinates:
  * y            (y) float64 800B 4.753e+06 4.753e+06 ... 4.752e+06 4.752e+06
  * x            (x) float64 800B 1.699e+05 1.699e+05 ... 1.709e+05 1.709e+05
    spatial_ref  int32 4B 25830
  * time         (time) datetime64[ns] 16B 2018-04-20T11:26:11.293000 2018-04...
Data variables:
    blue         (time, y, x) uint16 40kB 1662 1818 1948 2100 ... 357 381 390
    scl          (time, y, x) uint8 20kB 7 7 7 7 7 7 7 7 6 ... 4 4 4 4 4 4 4 4 4

In [138]:
data.blue == 0

<xarray.DataArray 'blue' (time: 2, y: 100, x: 100)> Size: 20kB
array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]])
Coordinates:
  * y            (y) float64 800B 4.753e+06 4.753e+06 ... 4.752e+06 4.752e+06
  * x            (x) float64 800B 1.699e+05 1.699e+05 ... 1.709e+05 1.709e+05
    spatial_ref  int32 4B 25830
  * time         (time) datetime64[ns] 16B 2018-04-20T11:26:11.293000 2018-04...

In [136]:
med = data.median(dim='time')
medskip = data.median(dim='time',skipna=True)

In [137]:
med

<xarray.Dataset> Size: 162kB
Dimensions:      (y: 100, x: 100)
Coordinates:
  * y            (y) float64 800B 4.753e+06 4.753e+06 ... 4.752e+06 4.752e+06
  * x            (x) float64 800B 1.699e+05 1.699e+05 ... 1.709e+05 1.709e+05
    spatial_ref  int32 4B 25830
Data variables:
    blue         (y, x) float64 80kB 1.188e+03 1.244e+03 ... 190.5 195.0
    scl          (y, x) float64 80kB 5.5 5.5 5.5 5.5 5.5 ... 2.0 2.0 2.0 2.0 2.0

In [139]:
med.rio.write_nodata(0)

AttributeError: 'RasterDataset' object has no attribute 'write_nodata'

In [127]:
np.unique(medskip.blue)

array([ 112.,  123.,  127., ..., 2845., 2884.,   nan], dtype=float32)

In [97]:
import numpy as np
for time in range(5):
    print(np.unique(data.blue[time]))

[   0   71   74 ... 3217 3300 3362]
[   0  123  130 ... 2991 3127 3142]


IndexError: index 2 is out of bounds for axis 0 with size 2

In [16]:
median = src_dataset['red'].median(dim='time',skipna=True).compute().fillna(0).astype('uint16')


/opt/homebrew/Caskroom/miniforge/base/envs/sentinel.mosaics/lib/python3.12/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/opt/homebrew/Caskroom/miniforge/base/envs/sentinel.mosaics/lib/python3.12/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/opt/homebrew/Caskroom/miniforge/base/envs/sentinel.mosaics/lib/python3.12/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/opt/homebrew/Caskroom/miniforge/base/envs/sentinel.mosaics/lib/python3.12/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/opt/homebrew/Caskroom/miniforge/base/envs/sentinel.mosaics/lib/python3.

<xarray.DataArray 'red' (y: 100, x: 100)> Size: 20kB
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)
Coordinates:
  * y            (y) float64 800B 4.859e+06 4.859e+06 ... 4.858e+06 4.858e+06
  * x            (x) float64 800B -1.408e+04 -1.408e+04 ... -1.31e+04 -1.31e+04
    spatial_ref  int32 4B 25830